In [1]:
import os
import re
import numpy as np
import pandas as pd
import shutil

## Remove Bott_Output If Exists Else Create Newly

In [2]:
if "Bott_Output" in next(os.walk(os.getcwd()))[1]:
    shutil.rmtree("Bott_Output")

os.mkdir("Bott_Output")

## Getting All Files List

In [3]:
cwd = os.getcwd() + "/"

directory_list = [cwd + "Bott/"]
file_list = []

while len(directory_list) != 0:

    for dl in directory_list:

        dir_list = next(os.walk(dl))[1]
        for i in range(len(dir_list)):
            dir_list[i] = dl + dir_list[i] + "/"
            
        directory_list.extend(dir_list)

        file_list.extend([dl+fl for fl in next(os.walk(dl))[2]])

        directory_list.remove(dl)

## Extracting Data From The Files

In [4]:
data = {"File_Name": [],
        "Serial": [],
        "Litho_1": [],
        "Litho_2": [],
        "Subject_Code": [],
        "Examiner_Code": [],
        "Paper_Serial_No": [],
        "Additional_Paper_No": [],
        "Examiner_Marks": [],
        "Head_Examiner_Marks": [],
        "Head_Examiner_Code": []
       }

split_length = [0, 32, 32, 3, 4, 4, 2, 3, 3, 4]
split = np.cumsum(split_length)

for fl in file_list:
    current_file = open(fl, "r")
    serial = 1
    for line in current_file:
        data["File_Name"].append(os.path.basename(fl)[:-4])
        data["Serial"].append(f"{serial:010d}")
        
        # iterate through all other keys excpet File_Name and Serial and set their values according to length
        key = list(data.keys())[2:]
        for i in range(0, len(key)):
            if key[i] == "Litho_1" or key[i] == "Litho_2":
                data[key[i]].append(line[split[i]:split[i+1]].replace(" ", "0"))    # replacing <space> with 0 in Litho
            else:
                data[key[i]].append(line[split[i]:split[i+1]])
        serial += 1

In [5]:
df = pd.DataFrame(data)

df.to_csv("Bott_Output/All_Bott.csv", index=False, sep=';')

## Seperate Duplicate Bott Rows

In [6]:
duplicate_bott = df[df.duplicated(["Litho_1", "Litho_2"])]

df = df.drop(duplicate_bott.index)

duplicate_bott.to_csv("Bott_Output/Duplicate_Bott.csv", index=False, sep=';')

## Adding Column Written_Marks

In [7]:
examiner_marks = df["Examiner_Marks"]
head_examiner_marks = df["Head_Examiner_Marks"]

marks = np.array([j if j != '   ' else i for i, j in zip(examiner_marks, head_examiner_marks)])

df["Written_Marks"] = marks

## Seperate Invalid Written_Marks

In [8]:
# is_numeric = lambda number: re.findall(r"([0][0-9][0-9])|([1][0][0])", number)
is_numeric = lambda number: re.findall(r"([0][0-9][0-9])|([1][0][0])|(^ [0-9][0-9])", number)

invalid_marks = [True if len(is_numeric(m)) == 0 else False for m in df["Written_Marks"]]

invalid_marks = df[invalid_marks]

df = df.drop(invalid_marks.index)

In [9]:
invalid_marks = invalid_marks[["File_Name", "Serial", "Examiner_Code", "Head_Examiner_Code", "Examiner_Marks", "Head_Examiner_Marks", "Written_Marks"]]

invalid_marks = invalid_marks.sort_values(["File_Name", "Serial"])

invalid_marks["File_Name"][invalid_marks["File_Name"].duplicated()] = ""

invalid_marks.to_excel("Bott_Output/Invalid_Marks_Report.xlsx", index=False)

## Seperate Unmatched Litho

In [10]:
unmatched_litho = df[df["Litho_1"] != df["Litho_2"]]

df = df.drop(unmatched_litho.index)

unmatched_litho["File_Name"][unmatched_litho["File_Name"].duplicated()] = ""

unmatched_litho.to_excel("Bott_Output/Unmatched_Litho_Bott_Report.xlsx", index=False)

## Final File

In [11]:
df.to_csv("Bott_Output/Bott.csv", index=False, sep=';')